# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [2]:
# import numpy and pandas
import numpy as np 
import pandas as pd
from scipy.stats import ttest_1samp

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [3]:
# Run this code:

salaries = pd.read_csv('../Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [4]:
# Your code here:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [6]:
# Your code here:
salaries.isnull().sum()

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [12]:
# Your code here:
salaries['Salary or Hourly'].value_counts()


Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [21]:
# Your code here:
salaries.pivot_table(index='Department', values='Name', aggfunc=({'Name':len})).head()

,Name
Department,
ADMIN HEARNG,39
ANIMAL CONTRL,81
AVIATION,1629
BOARD OF ELECTION,107
BOARD OF ETHICS,8


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [52]:
# Your code here:
from scipy.stats import ttest_1samp
#Hn: hourly workers =/= $30/hr
#Hå: hourly == $30/hr
# α = .05
hourly = [i for i in salaries['Hourly Rate'] if isinstance(i,float)]
ttest_1samp(hourly, 30, nan_policy='omit')

Ttest_1sampResult(statistic=20.619805785494147, pvalue=4.323024048614701e-92)

In [49]:
#pvalue is < a => reject null in favor of alt

In [55]:
salaries['Hourly Rate'].mean()

32.78855771628128

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [63]:
# Your code here:
#Hº: µ = 86000
#hå: µ > 86000
#α = 0.05

sample = salaries[salaries['Department']=='POLICE']['Annual Salary']

ttest_1samp(sample, 86000, nan_policy='omit')



Ttest_1sampResult(statistic=3.081997005712994, pvalue=0.00206034035510054)

In [64]:
salaries[salaries['Department']=='POLICE']['Annual Salary'].mean()

86486.41450313339

In [65]:
np.std(salaries[salaries['Department']=='POLICE']['Annual Salary'])

18271.546685177447

In [67]:
len(salaries[salaries['Department']=='POLICE']['Annual Salary'])

13414

Using the `crosstab` function, find the department that has the most hourly workers. 

In [80]:
# Your code here:
salaries[salaries['Salary or Hourly']=='Hourly'].groupby('Department').agg({'Name':len}).sort_values('Name', ascending=False).head()

,Name
Department,
STREETS & SAN,1862
WATER MGMNT,1513
OEMC,1273
AVIATION,1082
GENERAL SERVICES,765


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [83]:
# Your code here:
#Hº: µ = 35
#Hå: µ < 35
#α = 0.05

sample = salaries[salaries['Department']=='STREETS & SAN']['Hourly Rate']

ttest_1samp(sample, 35, nan_policy='omit')


Ttest_1sampResult(statistic=-9.567447887848152, pvalue=3.3378530564710738e-21)

In [ ]:
#pvalue is lower than α therefore we regect the null in favor of the alternate

# Challenge 3 - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [107]:
from scipy.stats import t
import scipy.stats

df = salaries[salaries['Salary or Hourly']=='Hourly']['Hourly Rate']
mean = df.mean()
err = scipy.stats.sem(df)

t.interval(0.95, df, loc=mean, scale=err)

(array([32.50633104, 32.51635598, 32.51417761, ..., 32.51417761,
        32.5163955 , 32.51676797]),
 array([33.07078439, 33.06075946, 33.06293782, ..., 33.06293782,
        33.06071993, 33.06034746]))

In [106]:
# Your code here:

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [108]:
# Your code here:
df = salaries[(salaries['Salary or Hourly']=='Salary')&(salaries['Department']=='POLICE')]['Annual Salary']
mean = df.mean()
err = scipy.stats.sem(df)

t.interval(0.95, df, loc=mean, scale=err)


(array([86177.08056118, 86177.08027413, 86177.08024141, ...,
        86177.07646443, 86177.08009305, 86177.08024141]),
 array([86795.74844509, 86795.74873214, 86795.74876486, ...,
        86795.75254183, 86795.74891322, 86795.74876486]))

# Challenge 4 - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [164]:
# Your code here:
import statsmodels as sm

count = salaries[salaries['Salary or Hourly']=='Hourly']['Hourly Rate']
mean = salaries[salaries['Salary or Hourly']=='Hourly']['Hourly Rate'].mean()
successes= [i for i in count if i>(mean*.75) and i<(mean*1.25)]

sm.stats.proportion.proportions_ztest(count=len(successes), nobs=len(count), value=.25)


(36.547667807210516, 1.9419256726570097e-292)

In [157]:
#no idea what this mean

3635

8022